In [1]:
import os, sys
from glob import glob
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from torchvision.models import alexnet

from tqdm import tqdm
import numpy as np

import cv2 as cv

from lbp import *

In [2]:
print(os.getcwd())
os.chdir("..")
print(os.getcwd())

D:\GitCloneProject\CE-AB-CLS\OtherSOTA
D:\GitCloneProject\CE-AB-CLS


In [3]:
data_dir = os.getcwd() + "\\Data set\\Original Form"
print(os.listdir(data_dir))

['Test', 'Train']


In [4]:
train_img_path = glob(data_dir + "\\Train\\*\\*")
test_img_path = glob(data_dir + "\\Test\\*\\*")

print(len(train_img_path))
print(len(test_img_path))

176
176


In [21]:
train_img_feature, train_label = [], []
test_img_feature, test_label = [], []

lbp = LBP()

dim = (256, 256)

for train_img, test_img in tqdm(zip(train_img_path, test_img_path)):
    tr_img = cv.imread(train_img)
    ts_img = cv.imread(test_img)
    
    resized_tr = cv2.resize(tr_img, dim, interpolation = cv2.INTER_AREA)
    resized_ts = cv2.resize(ts_img, dim, interpolation = cv2.INTER_AREA)
    
    lbp_tr_img = lbp.extract(resized_tr, gray = "grayscale")
    lbp_ts_img = lbp.extract(resized_ts, gray = "grayscale")
    
    train_img_feature.append(lbp_tr_img.flatten()/255)
    test_img_feature.append(lbp_ts_img.flatten()/255)
    
    train_label.append(0 if "Normal" in train_img else 1)
    test_label.append(0 if "Abnormal" in test_img else 1)

176it [03:33,  1.21s/it]


In [22]:
train_img_np = np.array(train_img_feature)
test_img_np = np.array(test_img_feature)

train_label_np = np.array(train_label)
test_label_np = np.array(test_label)

In [23]:
print(train_img_np.shape)
print(test_img_np.shape)
print(train_label_np.shape)
print(test_label_np.shape)

(176, 65536)
(176, 65536)
(176,)
(176,)


In [24]:
from sklearn import svm

clf = svm.SVC()
clf.fit(train_img_np, train_label_np)

y_pred = clf.predict(test_img_np)

In [25]:
from sklearn.metrics import classification_report

target_names = ['Normal', 'Abnormal']
print(classification_report(test_label_np, y_pred, target_names=target_names))

              precision    recall  f1-score   support

      Normal       0.00      0.00      0.00       120
    Abnormal       0.32      1.00      0.48        56

    accuracy                           0.32       176
   macro avg       0.16      0.50      0.24       176
weighted avg       0.10      0.32      0.15       176



d:\gitcloneproject\ce-ab-cls\envir\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\gitcloneproject\ce-ab-cls\envir\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\gitcloneproject\ce-ab-cls\envir\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
